# Processing 2014 crime data

In [1]:
import pandas as pd

data = pd.read_csv("crimes-in-chicago/Crimes_-_2014.csv")

data.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10581023,HZ329792,09/01/2014 08:00:00 AM,0000X E LAKE ST,1140,DECEPTIVE PRACTICE,EMBEZZLEMENT,OTHER,True,False,...,42.0,32,12,NaN,NaN,2014,03/01/2018 03:52:35 PM,NaN,NaN,NaN
1,10000348,HY190226,12/15/2014 04:00:00 PM,0000X E GRAND AVE,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,DEPARTMENT STORE,False,False,...,42.0,8,11,1176796.0,1903917.0,2014,02/10/2018 03:50:01 PM,41.891695,-87.626156,"(41.891694878, -87.626155832)"
2,10000465,HY190234,12/15/2014 04:30:00 PM,007XX N MICHIGAN AVE,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,DEPARTMENT STORE,False,False,...,42.0,8,11,1177303.0,1905177.0,2014,02/10/2018 03:50:01 PM,41.895141,-87.624256,"(41.895140898, -87.624255632)"
3,10000517,HY190259,11/15/2014 12:00:00 PM,082XX S ELLIS AVE,2820,OTHER OFFENSE,TELEPHONE THREAT,APARTMENT,False,True,...,8.0,44,26,1184330.0,1850584.0,2014,02/10/2018 03:50:01 PM,41.745172,-87.600159,"(41.745171682, -87.600158569)"
4,10000701,HY190349,06/20/2014 09:00:00 AM,004XX E ERIE ST,1130,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,OTHER,False,False,...,42.0,8,11,1179370.0,1904874.0,2014,02/10/2018 03:50:01 PM,41.894262,-87.616673,"(41.894262326, -87.616673471)"


In [2]:
data.size

6049802

In [3]:
data.columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location'],
      dtype='object')

#### Take out the needed columns

In [4]:
data = data[["ID", "Date", "Block", "Primary Type", "District", "Community Area", "Latitude", "Longitude"]]
data.head()

,ID,Date,Block,Primary Type,District,Community Area,Latitude,Longitude
0,10581023,09/01/2014 08:00:00 AM,0000X E LAKE ST,DECEPTIVE PRACTICE,1,32,NaN,NaN
1,10000348,12/15/2014 04:00:00 PM,0000X E GRAND AVE,DECEPTIVE PRACTICE,18,8,41.891695,-87.626156
2,10000465,12/15/2014 04:30:00 PM,007XX N MICHIGAN AVE,DECEPTIVE PRACTICE,18,8,41.895141,-87.624256
3,10000517,11/15/2014 12:00:00 PM,082XX S ELLIS AVE,OTHER OFFENSE,6,44,41.745172,-87.600159
4,10000701,06/20/2014 09:00:00 AM,004XX E ERIE ST,DECEPTIVE PRACTICE,18,8,41.894262,-87.616673


#### Remove null value in location

In [5]:
data2 = data[data['Date'].notnull() & data['Latitude'].notnull() & data['Longitude'].notnull() & data['Primary Type'].notnull()]

In [6]:
data2.size

2189688

#### Add Time period column and Datetime column

In [16]:
time_period = []

date_time = []

for index, row in data2.iterrows():
    date = pd.to_datetime(row["Date"])
    date_time.append(date)
    if 4<=date.hour<12:
        time_period.append("morning")
    elif 12<=date.hour<20:
        time_period.append("afternoon")
    else:
        time_period.append("evening")

In [32]:
data2["Time Period"] = time_period
data2["Datetime"] = date_time

C:\Users\newto\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\newto\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
data2 = data2[["Datetime", "Primary Type", "District", "Community Area", "Latitude", "Longitude","Time Period"]]

In [17]:
data2.head()

,Datetime,Primary Type,District,Community Area,Latitude,Longitude,Time Period
1,2014-12-15 16:00:00,DECEPTIVE PRACTICE,18,8,41.891695,-87.626156,afternoon
2,2014-12-15 16:30:00,DECEPTIVE PRACTICE,18,8,41.895141,-87.624256,afternoon
3,2014-11-15 12:00:00,OTHER OFFENSE,6,44,41.745172,-87.600159,afternoon
4,2014-06-20 09:00:00,DECEPTIVE PRACTICE,18,8,41.894262,-87.616673,morning
5,2014-03-15 09:00:00,DECEPTIVE PRACTICE,15,25,41.881737,-87.751484,morning


In [18]:
data2.to_pickle("crime_data_2014.pkl")